In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import numpy as np
from tensorflow import keras
from keras import Sequential
from keras import Input
from keras.layers import TextVectorization, Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from keras.models import load_model
from sklearn.model_selection import train_test_split

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcvar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jcvar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jcvar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def clean_text(text):
    text = strip_html(text)
    text = re.sub(r'[^A-Za-z0-9]+',' ',text)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [6]:
df = pd.read_csv('IMDB_Dataset.csv', encoding = 'Latin-1')
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))

x = df['Processed_Reviews']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

C:\Users\jcvar\AppData\Local\Temp\ipykernel_14232\387857404.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [7]:
vector_layer = TextVectorization(standardize='lower_and_strip_punctuation',
                                 max_tokens=10000,
                                 output_mode='int',
                                 output_sequence_length=30)
vector_layer.adapt(np.array(X_train))

# Save the TextVectorization layer's vocabulary
vocab = vector_layer.get_vocabulary()
with open('vocab.txt', 'w') as f:
    for item in vocab:
        f.write("%s\n" % item)

In [8]:
model = Sequential()
model.add(Input(shape=(1,), dtype=tf.string))
model.add(vector_layer)
model.add(Embedding(10001, 16))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_test), np.array(y_test)), epochs=20)

model.save('sentiment')

Epoch 1/20
1250/1250 [==============================] - 36s 13ms/step - loss: 0.5218 - accuracy: 0.7340 - val_loss: 0.4691 - val_accuracy: 0.7749
Epoch 2/20
1250/1250 [==============================] - 16s 13ms/step - loss: 0.4064 - accuracy: 0.8147 - val_loss: 0.4638 - val_accuracy: 0.7812
Epoch 3/20
1250/1250 [==============================] - 19s 15ms/step - loss: 0.3532 - accuracy: 0.8454 - val_loss: 0.4838 - val_accuracy: 0.7805
Epoch 4/20
1250/1250 [==============================] - 18s 15ms/step - loss: 0.3082 - accuracy: 0.8685 - val_loss: 0.5245 - val_accuracy: 0.7721
Epoch 5/20
1250/1250 [==============================] - 16s 13ms/step - loss: 0.2665 - accuracy: 0.8905 - val_loss: 0.5576 - val_accuracy: 0.7681
Epoch 6/20
1250/1250 [==============================] - 14s 11ms/step - loss: 0.2342 - accuracy: 0.9043 - val_loss: 0.6053 - val_accuracy: 0.7638
Epoch 7/20
1250/1250 [==============================] - 18s 14ms/step - loss: 0.2066 - accuracy: 0.9168 - val_loss: 0.6596 -

INFO:tensorflow:Assets written to: sentiment\assets


INFO:tensorflow:Assets written to: sentiment\assets


In [10]:
_, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(accuracy)

313/313 [==============================] - 2s 7ms/step - loss: 1.2188 - accuracy: 0.7442
0.7441999912261963


In [11]:
predictions =model.predict(["The movie was generally bad, the plot was boring and the characters badly interpreted"])
prediction = float(predictions[0][0])
class_name = 'negative' if prediction < 0.5 else 'positive'
if class_name == 'positive':
    confidence = (prediction - 0.5) / 0.5
else:
    confidence = (0.5 - prediction) / 0.5
print({'sentiment': class_name, 'confidence': confidence})

1/1 [==============================] - 0s 190ms/step
{'sentiment': 'negative', 'confidence': 0.9367949888110161}


In [13]:
from kerastuner.tuners import RandomSearch
from keras.layers import Bidirectional, LSTM

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def clean_text(text):
    text = strip_html(text)
    text = re.sub(r'[^A-Za-z0-9]+',' ',text)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df = pd.read_csv('IMDB_Dataset.csv', encoding = 'Latin-1')
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))

x = df['Processed_Reviews']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcvar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jcvar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jcvar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
C:\Users\jcvar\AppData\Local\Temp\ipykernel_14232\1021954690.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [14]:
vector_layer = TextVectorization(standardize='lower_and_strip_punctuation',
                                 max_tokens=10000,
                                 output_mode='int',
                                 output_sequence_length=100)
vector_layer.adapt(np.array(X_train))

vocab = vector_layer.get_vocabulary()
with open('vocab.txt', 'w') as f:
    for item in vocab:
        f.write("%s\n" % item)

In [15]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(1,), dtype=tf.string))
    model.add(vector_layer)
    model.add(Embedding(10000, hp.Int('embedding_dim', min_value=32, max_value=512, step=32)))
    model.add(Bidirectional(LSTM(hp.Int('LSTM_units', min_value=32, max_value=512, step=32))))
    model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [16]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=3,
    directory='model_dir',
    project_name='sentiment_analysis'
)

tuner.search(np.array(X_train), np.array(y_train), epochs=3, validation_data=(np.array(X_test), np.array(y_test)))

best_model = tuner.get_best_models(num_models=1)[0]

Trial 3 Complete [00h 05m 47s]
val_accuracy: 0.8637333313624064

Best val_accuracy So Far: 0.8673333326975504
Total elapsed time: 00h 18m 09s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [17]:
best_model.fit(np.array(X_train), np.array(y_train), epochs=20, validation_data=(np.array(X_test), np.array(y_test)))

# Save the model
best_model.save('sentiment')

_, accuracy = best_model.evaluate(np.array(X_test), np.array(y_test))
print(accuracy)

Epoch 1/20
1250/1250 [==============================] - 53s 40ms/step - loss: 0.1391 - accuracy: 0.9510 - val_loss: 0.3935 - val_accuracy: 0.8601
Epoch 2/20
1250/1250 [==============================] - 50s 40ms/step - loss: 0.0938 - accuracy: 0.9682 - val_loss: 0.5013 - val_accuracy: 0.8537
Epoch 3/20
1250/1250 [==============================] - 50s 40ms/step - loss: 0.0614 - accuracy: 0.9804 - val_loss: 0.6066 - val_accuracy: 0.8501
Epoch 4/20
1250/1250 [==============================] - 51s 41ms/step - loss: 0.0466 - accuracy: 0.9850 - val_loss: 0.6497 - val_accuracy: 0.8502
Epoch 5/20
1250/1250 [==============================] - 55s 44ms/step - loss: 0.0344 - accuracy: 0.9895 - val_loss: 0.7382 - val_accuracy: 0.8453
Epoch 6/20
1250/1250 [==============================] - 53s 42ms/step - loss: 0.0251 - accuracy: 0.9919 - val_loss: 0.8467 - val_accuracy: 0.8446
Epoch 7/20
1250/1250 [==============================] - 52s 42ms/step - loss: 0.0208 - accuracy: 0.9934 - val_loss: 0.8449 -

INFO:tensorflow:Assets written to: sentiment\assets


INFO:tensorflow:Assets written to: sentiment\assets


313/313 [==============================] - 5s 16ms/step - loss: 1.3798 - accuracy: 0.8395
0.8395000100135803


In [18]:
predictions =best_model.predict(["Quite horrible"])
prediction = float(predictions[0][0])
class_name = 'negative' if prediction < 0.5 else 'positive'
if class_name == 'positive':
    confidence = (prediction - 0.5) / 0.5
else:
    confidence = (0.5 - prediction) / 0.5
print({'sentiment': class_name, 'confidence': confidence})

1/1 [==============================] - 1s 656ms/step
{'sentiment': 'negative', 'confidence': 0.9999999815164298}


In [19]:
predictions =best_model.predict(["The movie was generally bad, the plot was boring and the characters badly interpreted"])
prediction = float(predictions[0][0])
class_name = 'negative' if prediction < 0.5 else 'positive'
if class_name == 'positive':
    confidence = (prediction - 0.5) / 0.5
else:
    confidence = (0.5 - prediction) / 0.5
print({'sentiment': class_name, 'confidence': confidence})

1/1 [==============================] - 0s 51ms/step
{'sentiment': 'negative', 'confidence': 0.9999993320166709}


In [ ]:
import pickle

# save the model to disk
filename = 'LSTM.pkl'
pickle.dump(best_model, open(filename, 'wb'))


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7a5b1e8f-5fb8-49ad-8f36-77068147d699' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>